In [10]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

In [21]:
df = pd.read_csv("chatgpt_features_full.csv",encoding='latin1')
print(df.columns.tolist())

df['FIPS'] = df['STATE_FIPS'].astype(str).str.zfill(2) + df['CZ_FIPS'].astype(str).str.zfill(3)
df['FIPS'] = df['FIPS'].astype(int)


['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH', 'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS', 'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME', 'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE', 'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE', 'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE', 'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE', 'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH', 'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON', 'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE', 'FIPS', 'customers_out', 'run_start_times', 'lead_time', 'BEGIN_DATE', 'END_DATE', 'BEGIN_HOUR', 'event_duration', 'avg_event_duration_by_type', 'season', 'region_outage_freq', 'TOR_F_SCALE_NUM', 'event_severity', 'dist_to_state_center', 

In [22]:
features = df[['customers_out', 'BEGIN_DATE_TIME', 'END_DATE_TIME',
    'BEGIN_LAT', 'BEGIN_LON', 'grid_density', 'grid_load_proxy', 'month_sin', 'is_weekday', 'region_avg_severity',
    'event_severity', 'EVENT_TYPE', 'avg_event_duration_by_type', 'region_outage_freq', 'FIPS']]

In [23]:
#change begin and end to datetime
features['BEGIN_DATE_TIME'] = pd.to_datetime(features['BEGIN_DATE_TIME'])
features['END_DATE_TIME'] = pd.to_datetime(features['END_DATE_TIME'])

#extract features
features['begin_month'] = features['BEGIN_DATE_TIME'].dt.month
features['begin_weekday'] = features['BEGIN_DATE_TIME'].dt.dayofweek  # Monday=0

#damage features are given in weird string format 
#features['DAMAGE_PROPERTY'] = features['DAMAGE_PROPERTY'].apply(parse_k).astype(int)
#features['DAMAGE_CROPS'] = features['DAMAGE_CROPS'].apply(parse_k).astype(int)

event_mapping = {event: idx for idx, event in enumerate(sorted(features['EVENT_TYPE'].unique()))}

def map_event_to_num(event):
    return event_mapping.get(event, -1)

features['event_type_num'] = features['EVENT_TYPE'].apply(map_event_to_num)

features = features.drop(columns=['BEGIN_DATE_TIME', 'END_DATE_TIME', 'EVENT_TYPE'])
print(features.columns.tolist())
features = features.dropna() #only one row contains NaNs

['customers_out', 'BEGIN_LAT', 'BEGIN_LON', 'grid_density', 'grid_load_proxy', 'month_sin', 'is_weekday', 'region_avg_severity', 'event_severity', 'avg_event_duration_by_type', 'region_outage_freq', 'FIPS', 'begin_month', 'begin_weekday', 'event_type_num']


C:\Users\jdsto\AppData\Local\Temp\ipykernel_7012\3175783415.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['BEGIN_DATE_TIME'] = pd.to_datetime(features['BEGIN_DATE_TIME'])
C:\Users\jdsto\AppData\Local\Temp\ipykernel_7012\3175783415.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['END_DATE_TIME'] = pd.to_datetime(features['END_DATE_TIME'])
C:\Users\jdsto\AppData\Local\Temp\ipykernel_7012\3175783415.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [24]:
features.to_csv("features_j.csv", index=False)